## 필요한 모듈 import

In [ ]:
!pip install keras-preprocessing
# !pip install --upgrade tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import cv2

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
# 텐서보드 콜백 생성, 손실 곡선 플로팅, 데이터 압축 해제, 훈련 이력 비교, 디렉토리 탐색, 예측 및 플로팅 등과 같은 함수 들고오기
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

--2024-06-13 10:33:17--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-06-13 10:33:17 (9.35 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
import os

def walk_through_dir_recursive(directory):
    for root, dirs, files in os.walk(directory):
        print(f"Currently in directory: {root}")
        print("Files:")
        for file in files:
            print(f"\t- {file}")
        print("Directories:")
        for dir in dirs:
            print(f"\t- {dir}")

## Load and Transform Data

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

In [ ]:
# Walk through each directory
import os

directory = '/content/drive/MyDrive/MobileNetV3_model_data/101_105'
contents = os.listdir(directory)
num_of_dirs = len([name for name in contents if os.path.isdir(os.path.join(directory, name))])
full_paths = [os.path.join(directory, item) for item in contents]

print("Contents of the directory:")
for item in full_paths:
    print(item)

print(f"\nNumber of directories: {num_of_dirs}")

Contents of the directory:
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자그라탕
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자구이
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자칩
/content/drive/MyDrive/MobileNetV3_model_data/101_105/가츠동
/content/drive/MyDrive/MobileNetV3_model_data/101_105/가지구이
/content/drive/MyDrive/MobileNetV3_model_data/101_105/가지
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감
/content/drive/MyDrive/MobileNetV3_model_data/101_105/갈비구이
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자샐러드
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자스프
/content/drive/MyDrive/MobileNetV3_model_data/101_105/고등어
/content/drive/MyDrive/MobileNetV3_model_data/101_105/고구마
/content/drive/MyDrive/MobileNetV3_model_data/101_105/고등어구이
/content/drive/MyDrive/MobileNetV3_model_data/101_105/감자튀김
/content/drive/MyDrive/MobileNetV3_model_data/101_105/건자두
/content/drive/MyDrive/Mob

In [ ]:
import os
import pandas as pd

#### 음식 메뉴 10개만 먼저 하기 ####

# 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/MobileNetV3_model_data/101_105'
contents = os.listdir(directory)

full_paths = [os.path.join(directory, item) for item in contents]

# 디렉토리의 모든 파일(음식) 리스트 저장
file_names = os.listdir(directory_path)

print(file_names)

# 디렉토리로부터 이미지 로드 [근데 이미지 있는 파일까지 안 내려]
data_list = []
label_list = []

for path in full_paths[:10]:
    class_name = os.path.basename(path)
    print(class_name, " 시작")
    if os.path.isdir(path):
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            data_list.append(img_path)
            label_list.append(class_name)

# 파일 경로와 라벨을 Series로 변환
filepaths = pd.Series(data_list, name='Filepath').astype(str)
labels = pd.Series(label_list, name='Label')

# 파일 경로와 라벨을 DataFrame으로 합침
image_df = pd.concat([filepaths, labels], axis=1)

# 결과 확인
print(image_df.head())

In [ ]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/MobileNetV3_model_data/...,감자그라탕
1,/content/drive/MyDrive/MobileNetV3_model_data/...,감자그라탕
2,/content/drive/MyDrive/MobileNetV3_model_data/...,감자그라탕
3,/content/drive/MyDrive/MobileNetV3_model_data/...,감자그라탕
4,/content/drive/MyDrive/MobileNetV3_model_data/...,감자그라탕
...,...,...
9995,/content/drive/MyDrive/MobileNetV3_model_data/...,감자스프
9996,/content/drive/MyDrive/MobileNetV3_model_data/...,감자스프
9997,/content/drive/MyDrive/MobileNetV3_model_data/...,감자스프
9998,/content/drive/MyDrive/MobileNetV3_model_data/...,감자스프


In [ ]:
import pandas as pd

# image_df를 CSV 파일로 저장
image_df.to_csv('/content/drive/MyDrive/MobileNetV3_model_data/image_data_10.csv', index=False)

## Data Preprocessing

In [ ]:
# Define the resize and rescale layer
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(224, 224),
    layers.experimental.preprocessing.Rescaling(1./255),
])

# Function to preprocess images
def preprocess_image(image_path):
    print(image_path)
    img = tf.keras.preprocessing.image.load_img(image_path)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Expand dims to make it (1, height, width, channels)
    img_array = resize_and_rescale(img_array)  # Apply resizing and rescaling
    return img_array

In [ ]:
# 이미지와 라벨 배열 저장 후 train_test_data one_hot encoding 하기
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Preprocess images and create dataset
preprocessed_images = np.vstack([preprocess_image(path) for path in data_list])
image_labels = np.array(label_list)

# Save preprocessed images and labels as .npy files
# np.save('/content/drive/MyDrive/MobileNetV3_model_data/data/preprocessed_images.npy', preprocessed_images)
# np.save('/content/drive/MyDrive/MobileNetV3_model_data/data/image_labels.npy', image_labels)

# Ensure image_labels are properly encoded as integers
unique_labels = np.unique(image_labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
indexed_labels = np.array([label_to_index[label] for label in image_labels])

# Convert labels to one-hot encoding
one_hot_labels = to_categorical(indexed_labels, num_classes=10)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(preprocessed_images, one_hot_labels, test_size=0.2, random_state=42)

# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)

# Check shapes
print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Validation images shape: {val_images.shape}")
print(f"Validation labels shape: {val_labels.shape}")

NameError: name 'data_list' is not defined

In [ ]:
# 저장한 이미지 배열 로드해서 사용하기
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# Load the saved .npy files
preprocessed_images = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data/preprocessed_images.npy')
image_labels = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data/image_labels.npy')


# Ensure image_labels are properly encoded as integers
unique_labels = np.unique(image_labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
indexed_labels = np.array([label_to_index[label] for label in image_labels])

# Convert labels to one-hot encoding
one_hot_labels = to_categorical(indexed_labels, num_classes=10)

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(preprocessed_images, one_hot_labels, test_size=0.2, random_state=42)

# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)

# Check shapes
print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Validation images shape: {val_images.shape}")
print(f"Validation labels shape: {val_labels.shape}")


Train images shape: (8000, 224, 224, 3)
Train labels shape: (8000, 10)
Validation images shape: (2000, 224, 224, 3)
Validation labels shape: (2000, 10)


In [ ]:
# train_generator = ImageDataGenerator(
#     preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
#     validation_split=0.2
# )

# test_generator = ImageDataGenerator(
#     preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input
# )

In [ ]:
# # Split the data into three categories.
# train_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='Filepath',
#     y_col='Label',
#     target_size=(224, 224),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=32,
#     shuffle=True,
#     seed=42,
#     subset='training'
# )

# val_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='Filepath',
#     y_col='Label',
#     target_size=(224, 224),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=32,
#     shuffle=True,
#     seed=42,
#     subset='validation'
# )

# test_images = test_generator.flow_from_dataframe(
#     dataframe=test_df,
#     x_col='Filepath',
#     y_col='Label',
#     target_size=(224, 224),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=32,
#     shuffle=False
# )

Found 6400 validated image filenames belonging to 10 classes.
Found 1600 validated image filenames belonging to 10 classes.
Found 2000 validated image filenames belonging to 10 classes.


## Training the model

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)

pretrained_model.trainable = False

4334752/4334752 [==============================] - 1s 0us/step


In [ ]:
# Create checkpoint callback
# checkpoint_path = "/content/drive/MyDrive/MobileNetV3_model_data/checkpoint"
# checkpoint_callback = ModelCheckpoint(checkpoint_path,
#                                       save_weights_only=True,
#                                       monitor="val_accuracy",
#                                       save_best_only=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/MobileNetV3_model_data/checkpoint',
                                                         monitor='val_accuracy',
                                                         verbose=1,
                                                         save_best_only=True)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

In [ ]:
# 1안
# Create the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = pretrained_model(inputs, training=False)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #0.0001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# load_model()해서 이어서 학습

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", "food_classification"),
        checkpoint_callback,
    ]
)

Saving TensorBoard log files to: training_logs/food_classification/20240612-080048
Epoch 1/200
249/250 [============================>.] - ETA: 0s - loss: 2.3011 - accuracy: 0.1273
Epoch 1: val_accuracy did not improve from 0.36400
250/250 [==============================] - 12s 29ms/step - loss: 2.3008 - accuracy: 0.1277 - val_loss: 2.2508 - val_accuracy: 0.1810
Epoch 2/200
247/250 [============================>.] - ETA: 0s - loss: 2.2352 - accuracy: 0.1742
Epoch 2: val_accuracy did not improve from 0.36400
250/250 [==============================] - 5s 19ms/step - loss: 2.2343 - accuracy: 0.1748 - val_loss: 2.1934 - val_accuracy: 0.2375
Epoch 3/200
250/250 [==============================] - ETA: 0s - loss: 2.1723 - accuracy: 0.2096
Epoch 3: val_accuracy did not improve from 0.36400
250/250 [==============================] - 5s 19ms/step - loss: 2.1723 - accuracy: 0.2096 - val_loss: 2.1365 - val_accuracy: 0.2430
Epoch 4/200
247/250 [============================>.] - ETA: 0s - loss: 2.118

In [ ]:
# 저장된 모델 가중치 로드
model.load_weights('/content/drive/MyDrive/MobileNetV3_model_data/checkpoint')

# 모델 재학습
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", "food_classification"),
        checkpoint_callback,
    ]
)

Saving TensorBoard log files to: training_logs/food_classification/20240612-084029
Epoch 1/30
249/250 [============================>.] - ETA: 0s - loss: 1.2908 - accuracy: 0.5526
Epoch 1: val_accuracy did not improve from 0.54700
250/250 [==============================] - 5s 19ms/step - loss: 1.2906 - accuracy: 0.5524 - val_loss: 1.2763 - val_accuracy: 0.5460
Epoch 2/30
247/250 [============================>.] - ETA: 0s - loss: 1.2905 - accuracy: 0.5515
Epoch 2: val_accuracy did not improve from 0.54700
250/250 [==============================] - 5s 19ms/step - loss: 1.2896 - accuracy: 0.5516 - val_loss: 1.2755 - val_accuracy: 0.5450
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 1.2874 - accuracy: 0.5575
Epoch 3: val_accuracy did not improve from 0.54700
250/250 [==============================] - 5s 19ms/step - loss: 1.2879 - accuracy: 0.5566 - val_loss: 1.2716 - val_accuracy: 0.5425
Epoch 4/30
247/250 [============================>.] - ETA: 0s - loss: 1.2886 - a

In [ ]:
# 1안 모델 이어서

# Define additional layers
additional_layers = [
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
]
# Create a new input layer
inputs = tf.keras.Input(shape=(224, 224, 3))

# Pass the input through the pretrained model
x = pretrained_model(inputs, training=False)

# Add additional layers
for layer in additional_layers:
    x = layer(x)

# Define the output layer
outputs = layers.Dense(10, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Load weights
model.load_weights('/content/drive/MyDrive/MobileNetV3_model_data/checkpoint')

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", "food_classification"),
        checkpoint_callback,
    ]
)


Saving TensorBoard log files to: training_logs/food_classification/20240613-063606
Epoch 1/200


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

### 2안 (돌리는중)

In [ ]:
##########모델 설계#########
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_large',
                                                         monitor='val_accuracy',
                                                         verbose=1,
                                                         save_best_only=True)

batch_size = 16

pretrained_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
)

pretrained_model.trainable = False

# Create the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = pretrained_model(inputs, training=False)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), #0.0001 #optimizer=tf.keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=500,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", "food_classification"),
        checkpoint_callback,
        # lr_scheduler, #이거빼도 될듯
    ]
)

In [ ]:
#########모델 추가학습#########
import tensorflow as tf
from tensorflow.keras import layers, models

# 모델 체크포인트 경로
checkpoint_path = '/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_large'

# 저장된 모델을 로드
model = tf.keras.models.load_model(checkpoint_path)

# 이전 모델 설정 그대로 사용하여 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 추가 학습을 위한 새로운 콜백 설정 (필요에 따라)
additional_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                                    monitor='val_accuracy',
                                                                    verbose=1,
                                                                    save_best_only=True)

# 추가 학습 수행
additional_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2000,  # 필요한 만큼의 에포크 설정
    callbacks=[
        early_stopping,
        create_tensorboard_callback("additional_training_logs", "food_classification"),
        additional_checkpoint_callback,
    ]
)


Saving TensorBoard log files to: additional_training_logs/food_classification/20240613-054453
Epoch 1/2000
249/250 [============================>.] - ETA: 0s - loss: 1.0297 - accuracy: 0.6402
Epoch 1: val_accuracy improved from -inf to 0.62300, saving model to /content/drive/MyDrive/MobileNetV3_model_data/checkpoint_large
250/250 [==============================] - 30s 104ms/step - loss: 1.0297 - accuracy: 0.6401 - val_loss: 1.0928 - val_accuracy: 0.6230
Epoch 2/2000
249/250 [============================>.] - ETA: 0s - loss: 1.0131 - accuracy: 0.6496
Epoch 2: val_accuracy did not improve from 0.62300
250/250 [==============================] - 7s 28ms/step - loss: 1.0132 - accuracy: 0.6496 - val_loss: 1.1030 - val_accuracy: 0.6155
Epoch 3/2000
250/250 [==============================] - ETA: 0s - loss: 1.0183 - accuracy: 0.6394
Epoch 3: val_accuracy did not improve from 0.62300
250/250 [==============================] - 7s 28ms/step - loss: 1.0183 - accuracy: 0.6394 - val_loss: 1.1014 - v

In [ ]:
model.save('/content/drive/MyDrive/MobileNetV3_model_data/model/large_mobileNet3_model.h5')

ValueError: Unable to synchronously create dataset (name already exists)

### 모델 튜닝

In [ ]:
!pip install tf-models-official

#### 튜닝을 위한 데이터 준비

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models
import numpy as np

# 데이터 전처리 함수
def custom_preprocess_function(img):
    return tf.keras.applications.mobilenet_v3.preprocess_input(img)

def preprocess_images(images):
    return np.array([custom_preprocess_function(image) for image in images])

# 데이터 로드 및 전처리 (데이터는 사용자가 제공한 경로에서 로드)
train_datagen = ImageDataGenerator(validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/MobileNetV3_model_data/101_105',
    target_size=(224, 224),  # 이미지가 224x224로 리사이즈 됩니다.
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=False
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/MobileNetV3_model_data/101_105',
    target_size=(224, 224),  # 이미지가 224x224로 리사이즈 됩니다.
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# 배치로 데이터 가져오기 및 전처리
train_images, train_labels = next(train_generator)
train_images = preprocess_images(train_images)

validation_images, validation_labels = next(validation_generator)
validation_images = preprocess_images(validation_images)

print(f"Train images shape: {train_images.shape}")
print(f"Validation images shape: {validation_images.shape}")


Found 44800 images belonging to 56 classes.
Found 11198 images belonging to 56 classes.
Train images shape: (32, 224, 224, 3)
Validation images shape: (32, 224, 224, 3)


In [ ]:

# 학습률 설정
learning_rate = 0.0001

# 최적화기 설정
optimizer_names = [
    ('SGD', SGD(learning_rate=learning_rate)),
    ('SGD with momentum', SGD(learning_rate=learning_rate, momentum=0.9)),
    ('Adam', Adam(learning_rate=learning_rate)),
    ('RMSprop', RMSprop(learning_rate=learning_rate)),
    ('Adagrad', Adagrad(learning_rate=learning_rate))
]

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# 각 옵티마이저로 모델 학습 및 평가
for optimizer_name, optimizer in optimizer_names:
    print(f"Using optimizer: {optimizer_name}")

    # MobileNetV3 모델 로드 및 정의
    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # 모델의 최상위 층 추가
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(train_generator.num_classes, activation='softmax')
    ])

    # 모델 컴파일
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # 모델 학습
    model.fit(
        train_images, train_labels,
        validation_data=(validation_images, validation_labels),
        epochs=50,
        callbacks=[early_stopping, reduce_lr]
    )

    # 모델 평가
    loss, accuracy = model.evaluate(validation_images, validation_labels)
    print(f"Test accuracy with {optimizer_name}: {accuracy:.4f}\n")

Using optimizer: SGD
Epoch 1/50
1/1 [==============================] - 12s 12s/step - loss: 3.6070 - accuracy: 0.0625 - val_loss: 3.2487 - val_accuracy: 0.1562 - lr: 1.0000e-04
Epoch 2/50
1/1 [==============================] - 0s 119ms/step - loss: 3.4479 - accuracy: 0.1250 - val_loss: 3.2404 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 3/50
1/1 [==============================] - 0s 121ms/step - loss: 3.7084 - accuracy: 0.0625 - val_loss: 3.2326 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 4/50
1/1 [==============================] - 0s 119ms/step - loss: 3.5926 - accuracy: 0.0625 - val_loss: 3.2250 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 5/50
1/1 [==============================] - 0s 119ms/step - loss: 3.5115 - accuracy: 0.1562 - val_loss: 3.2171 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 6/50
1/1 [==============================] - 0s 126ms/step - loss: 3.6271 - accuracy: 0.0625 - val_loss: 3.2093 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 7/50
1/1 [======================

#### 전처리 먼저

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


# 데이터 전처리 함수
def custom_preprocess_function(img):
    return tf.keras.applications.mobilenet_v3.preprocess_input(img)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_images(directory, target_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    num_classes = len(class_names)
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, fname)
                    img = load_img(img_path, target_size=target_size)
                    img_array = img_to_array(img)
                    img_array = custom_preprocess_function(img_array)
                    images.append(img_array)
                    labels.append(class_index)
    images = np.array(images)
    labels = tf.keras.utils.to_categorical(labels, num_classes)
    return images, labels

# 데이터 경로 설정
train_dir = '/content/drive/MyDrive/MobileNetV3_model_data/101_105_crop'

# 데이터 로드 및 전처리
images, labels = load_and_preprocess_images(train_dir)

#train과 test 분리
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# 분할된 데이터셋 확인
print(f"훈련 이미지 수: {train_images.shape[0]}")
print(f"훈련 레이블 수: {train_labels.shape[0]}")
print(f"테스트 이미지 수: {test_images.shape[0]}")
print(f"테스트 레이블 수: {test_labels.shape[0]}")

In [ ]:
import numpy as np

# 이미지와 레이블을 NumPy 배열로 저장
np.save('/content/drive/MyDrive/MobileNetV3_model_data/opt_images_10.npy', images)
np.save('/content/drive/MyDrive/MobileNetV3_model_data/opt_labels_10.npy', labels)

print("데이터 저장이 완료되었습니다.")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# 저장된 NumPy 배열 데이터 로드
loaded_images = np.load('/content/drive/MyDrive/MobileNetV3_model_data/opt_images_10.npy')
loaded_labels = np.load('/content/drive/MyDrive/MobileNetV3_model_data/opt_labels_10.npy')

print("데이터 로드가 완료되었습니다.")

데이터 로드가 완료되었습니다.


In [ ]:
#train과 test 분리
train_images, test_images, train_labels, test_labels = train_test_split(loaded_images, loaded_labels, test_size=0.2, random_state=42, stratify=loaded_labels)

In [ ]:
# 분할된 데이터셋 확인
print(f"훈련 이미지 수: {train_images.shape}")
print(f"훈련 레이블 수: {train_labels.shape}")
print(f"테스트 이미지 수: {test_images.shape}")
print(f"테스트 레이블 수: {test_labels.shape}")

훈련 이미지 수: (7991, 224, 224, 3)
훈련 레이블 수: (7991, 10)
테스트 이미지 수: (1998, 224, 224, 3)
테스트 레이블 수: (1998, 10)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import pandas as pd

# 학습률 설정
learning_rate = 0.0001

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['optimizer', 'loss', 'accuracy'])

# 최적화기 설정
optimizer_names = [
    ('SGD', SGD(learning_rate=learning_rate)),
    ('SGD with momentum', SGD(learning_rate=learning_rate, momentum=0.9)),
    ('Adam', Adam(learning_rate=learning_rate)),
    ('RMSprop', RMSprop(learning_rate=learning_rate)),
    ('Adagrad', Adagrad(learning_rate=learning_rate))
]

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# 각 옵티마이저로 모델 학습 및 평가
for optimizer_name, optimizer in optimizer_names:
    print(f"Using optimizer: {optimizer_name}")

    # MobileNetV3 모델 로드 및 정의
    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )

    # 모델의 최상위 층 추가
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(train_labels.shape[1], activation='softmax')
    ])

    # 모델 컴파일
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Checkpoint 콜백 설정
    checkpoint_callback = ModelCheckpoint(
        filepath=f'/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/{optimizer_name}_checkpoint.h5',
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max'
    )

    # 모델 학습
    model.fit(
        train_images, train_labels,
        validation_data=(test_images, test_labels),
        epochs=50,
        callbacks=[early_stopping, reduce_lr, checkpoint_callback]
    )

    # 모델 평가
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f"Test accuracy with {optimizer_name}: {accuracy:.4f}\n")

    # # Store the results in the DataFrame
    # results_df = results_df.append({
    #     'optimizer': optimizer_name,
    #     'loss': loss,
    #     'accuracy': accuracy
    # }, ignore_index=True)

# 결과 저장
# results_df.to_csv('/content/drive/MyDrive/MobileNetV3_model_data/results.csv', index=False)


Using optimizer: SGD
4334752/4334752 [==============================] - 0s 0us/step


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-e23ab797a32a>", line 38, in <cell line: 27>
    model = models.Sequential([
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 204, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/sequential.py", line 135, in __init__
    self.add(layer)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/trackable/base.py", line 204, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return f

TypeError: object of type 'NoneType' has no len()

In [ ]:
model.save('/content/drive/MyDrive/MobileNetV3_model_data/model/sgd_mobile_small_model.hd5')

In [ ]:
##추가학습
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad

# 학습률 설정
learning_rate = 0.0001

# 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint.h5',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)


# 저장된 모델 로드
model_path = '/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint.h5'
model = tf.keras.models.load_model(model_path)

#optimizet를 SGD [ loss: 0.4873 - accuracy: 0.8453 - val_loss: 0.2824 - val_accuracy: 0.9254 - lr: 1.0000e-04]로 200epoch 돌리고 그이후부터 momentum?그거 추가함
# SGD(learning_rate=learning_rate) : 현재의 기울기만 반영
# SGD(learning_rate=learning_rate, momentum=0.9) : 이전 업데이트를 영향 받아 반영
# 모델 컴파일
optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 추가 학습
model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    epochs=100,  # 추가 학습할 에포크 수
    callbacks=[early_stopping, reduce_lr, checkpoint_callback]
)

# 모델 평가
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy after additional training with SGD with momentum: {accuracy:.4f}\n")


Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.0600 - accuracy: 0.9811
Epoch 1: val_accuracy improved from -inf to 0.98198, saving model to /content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 130s 488ms/step - loss: 0.0600 - accuracy: 0.9811 - val_loss: 0.0569 - val_accuracy: 0.9820 - lr: 1.0000e-04
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.0576 - accuracy: 0.9825
Epoch 2: val_accuracy did not improve from 0.98198
250/250 [==============================] - 124s 496ms/step - loss: 0.0576 - accuracy: 0.9825 - val_loss: 0.0568 - val_accuracy: 0.9815 - lr: 1.0000e-04
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.0575 - accuracy: 0.9826
Epoch 3: val_accuracy did not improve from 0.98198
250/250 [==============================] - 118s 470ms/step - loss: 0.0575 - accuracy: 0.9826 - val_loss: 0.0561 - val_accuracy: 0.9815 - lr: 1.0000e-04
Epoch 4/100
250/250 [==============================] - ETA: 0s - loss: 0.0609 - accuracy: 0.9807
Epoch 4: val_accuracy did not improve from 0.98198
250/250 [==============================] - 117s 469ms/step - loss: 0.0609 - accuracy: 0.9807 - val_l

In [ ]:
#validation 평가 (이거 지금 갯수 다 안들어감.)
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

# 데이터 전처리 함수
def custom_preprocess_function(img):
    return tf.keras.applications.mobilenet_v3.preprocess_input(img)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_images(directory, target_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    num_classes = len(class_names)
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, fname)
                    try:
                        img = load_img(img_path, target_size=target_size)
                        img_array = img_to_array(img)
                        img_array = custom_preprocess_function(img_array)
                        images.append(img_array)
                        labels.append(class_index)
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
                        continue
    images = np.array(images)
    labels = to_categorical(labels, num_classes)
    return images, labels

# 데이터 경로 설정
validation_dir = '/content/drive/MyDrive/food_data/Crop/validation/101'

# 데이터 로드 및 전처리
validation_images, validation_labels = load_and_preprocess_images(validation_dir)

# 모델 로드
model = load_model('/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint.h5')

# 모델 평가
loss, accuracy = model.evaluate(validation_images, validation_labels)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

In [ ]:
#히스토리 그래프 그리기
import numpy as np
import matplotlib.pyplot as plt

# 히스토리 로드
history = np.load('/content/drive/MyDrive/MobileNetV3_model_data/history.npy', allow_pickle=True).item()

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 손실값 그래프
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 101-110

In [ ]:
import os

# 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/MobileNetV3_model_data/101_110'

# 하위 디렉토리 이름과 갯수 출력 함수
def print_subdirectory_names_and_count(directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return

    subdirectories = [name for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]
    total_subdirectories = len(subdirectories)

    print(f"Total number of subdirectories: {total_subdirectories}")
    print("Subdirectory names:")
    for subdirectory in subdirectories:
        print(subdirectory)

# 함수 호출
print_subdirectory_names_and_count(directory_path)

Total number of subdirectories: 106
Subdirectory names:
귤
군밤
군고구마
국수
구아바
과일채소샐러드
곶감
고추장아찌
고추
고르곤졸라피자
고르곤졸라
감자튀김
거봉포도
건자두
건포도
게맛살
게장
경단
고구마
고등어
고등어구이
가지
가지구이
가츠동
갈비구이
감
감자구이
감자그라탕
감자샐러드
감자스프
감자칩
당근케이크
당근
닭훈제구이
닭찜
닭살채소볶음
닭살채소두반장볶음
닭날개양념구이
닭날개
닭고기볶음
닭강정
닭갈비
닭가슴살시저샐러드
닭가슴살샐러드
닭가슴살
달걀후라이
달걀샌드위치
달걀비빔밥
달걀볶음밥
달걀
단호박
단감
다크초콜릿
다시마
누텔라
녹색피망
냉면
나시고랭
꿀
꼬막
껌
까망베르치즈
까르보나라
김치찌개
김치볶음밥
김치국
김밥
김마끼
김
그라탕
두부튀김
두부
돼지고기스테이크
돼지고기구이
돼지고기고추장불고기
돼지감자
돼지갈비찜
도리아
도넛
데친두부
데리야끼치킨
대파
대추
마
리코타치즈
리조또
롤케이크
로메인상추
레몬
랍스타
라자냐
라면
똠양꿍
떡국

In [ ]:
import os
import pandas as pd

# 두 디렉토리 경로 설정
dir_101 = '/content/drive/MyDrive/food_data/Crop/validation/101'
dir_104 = '/content/drive/MyDrive/food_data/Crop/validation/104'

# CSV 파일 경로 설정
csv_file = '/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110/class_names.csv'

# 디렉토리 내의 파일 리스트 가져오기
files_101 = os.listdir(dir_101)
files_104 = os.listdir(dir_104)

# 두 디렉토리의 파일 리스트 합치기
all_files = files_101 + files_104

# CSV 파일에서 클래스 이름 읽어오기 (인코딩 설정)
try:
    class_names_df = pd.read_csv(csv_file, encoding='utf-8')
except UnicodeDecodeError:
    class_names_df = pd.read_csv(csv_file, encoding='latin1')

class_names = class_names_df.iloc[:, 0].tolist()  # 첫 번째 열의 값들을 리스트로 변환

# 파일 리스트에서 확장자를 제거하여 클래스 이름 추출
extracted_class_names = [os.path.splitext(file)[0] for file in all_files]

# CSV 파일에 없는 클래스 이름 찾기
missing_class_names = set(extracted_class_names) - set(class_names)

# 결과 출력
print("CSV 파일에 없는 클래스 이름들:")
for name in missing_class_names:
    print(name)


CSV 파일에 없는 클래스 이름들:
과일채소샐러드
고구마
달걀비빔밥
닭살채소두반장볶음
꿀
닭찜
까르보나라
김치볶음밥
군밤
고르곤졸라
고추
감자샐러드
나시고랭
가지
감자튀김
냉면
감자스프
노랑파프리카
김치찌개
다시마
고추장아찌
거봉포도
고르곤졸라피자
녹색피망
감자구이
감자볶음
그라탕
건포도
단호박
달걀후라이
닭가슴살시저샐러드
닭날개양념구이
김치국
똠양꿍
감자그라탕
귤
닭가슴살
닭가슴살샐러드
껌
곶감
가지구이
경단
달걀샌드위치
가츠동
갈비구이
구아바
김
당근케이크
감
고등어구이
달걀
다크초콜릿
당근
감자칩
단감
김마끼
김밥
달걀볶음밥
닭날개
게장
게맛살
꼬막
닭살채소볶음
누텔라
닭훈제구이
군고구마
닭고기볶음
까망베르치즈
건자두
닭강정
김구이
닭갈비
고등어
국수


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import csv
from tqdm import tqdm

# 데이터 전처리 함수
def custom_preprocess_function(img):
    return tf.keras.applications.mobilenet_v3.preprocess_input(img)

# 클래스 이름과 인덱스를 CSV로 저장하는 함수
def save_class_names_to_csv(class_names, csv_path):
    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Class Index', 'Class Name'])
        for index, name in enumerate(class_names):
            writer.writerow([index, name])

# 데이터 로드 및 전처리 함수
def load_and_preprocess_images_in_batches(directory, target_size=(224, 224), batch_size=128, save_dir='/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110'):
    class_names = sorted(os.listdir(directory))
    num_classes = len(class_names)
    image_paths = []

    # 클래스 이름과 인덱스를 CSV로 저장
    # csv_path = os.path.join(save_dir, 'class_names.csv')
    # save_class_names_to_csv(class_names, csv_path)

    # 모든 이미지 경로를 수집
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, fname)
                    image_paths.append((img_path, class_index))

    # 총 이미지 수
    total_images = len(image_paths)
    print('총 이미지 수 :', total_images)
    images_per_batch = batch_size * num_classes
    num_batches = (total_images + images_per_batch - 1) // images_per_batch  # 배치 수 계산

    for i in tqdm(range(num_batches), desc="Processing batches"):

        if i < 4: #3까지 다운받은 상태
            continue

        start = i * images_per_batch
        end = min(start + images_per_batch, total_images)
        batch_paths = image_paths[start:end]

        images = []
        labels = []

        for img_path, class_index in batch_paths:
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            img_array = custom_preprocess_function(img_array)
            images.append(img_array)
            labels.append(class_index)

        print(f"{i}번째 이미지 수 :", len(images))
        images = np.array(images)
        labels = tf.keras.utils.to_categorical(labels, num_classes)

        # 배치 데이터를 .npy 파일로 저장
        np.save(os.path.join(save_dir, f'images_validation_{i}.npy'), images)
        np.save(os.path.join(save_dir, f'labels_validation_{i}.npy'), labels)

# 데이터 경로 설정
train_dir = '/content/drive/MyDrive/MobileNetV3_model_data/101_110'
save_dir = '/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110'
os.makedirs(save_dir, exist_ok=True)

# 데이터 로드 및 배치로 저장
batch_size = 128
load_and_preprocess_images_in_batches(train_dir, batch_size=batch_size, save_dir=save_dir)

print("Data saving completed.")


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split

# 학습률 및 콜백 설정
learning_rate = 0.0001
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint_10.h5',#######0번째 데이터로 학습 시작
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# 저장된 모델 로드
model_path = '/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint_9.h5'############이전 데이터 한걸로 바꿔주기
base_model = tf.keras.models.load_model(model_path)

#첫번째에만 실시 클래스 수가 10개와 106개로 달랐기 때문에
# x = base_model.layers[-2].output  # 마지막 Dense 레이어 직전의 레이어를 가져옴
# new_output = Dense(106, activation='softmax', name='new_dense_output')(x)  # 새로운 클래스 수로 Dense 레이어 추가
# new_model = Model(inputs=base_model.input, outputs=new_output)

#두번째 데이터셋부터 output수 같으므로 실행해도 됨
#4는 확실히 load_model로 했는데 3이 기억이 안나..
new_model = base_model

# 모델 컴파일
optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
new_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# # .npy 파일 로드
# data_images = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110/images_batch_2.npy') ##############학습할때마다 업데이트
# data_labels = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110/labels_batch_2.npy') ##############학습할때마다 업데이트

# # 데이터를 학습용과 테스트용으로 나누기 (80% 학습용, 20% 테스트용)
# train_images, test_images, train_labels, test_labels = train_test_split(data_images, data_labels, test_size=0.2, random_state=42)

#마지막 검증셋 설정
train_images = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110/images_batch_2.npy')
train_labels = np.load('/content/drive/MyDrive/MobileNetV3_model_data/data_batches_101_110/labels_batch_2.npy')

# 추가 학습
new_model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),  # 필요에 따라 검증 데이터 설정
    epochs=200,  # 추가 학습할 에포크 수
    callbacks=[early_stopping, reduce_lr, checkpoint_callback]
)

# 모델 평가 (필요 시 검증 데이터로 평가)
loss, accuracy = new_model.evaluate(test_images, test_labels)
print(f"Test accuracy after additional training with SGD with momentum: {accuracy:.4f}")


Epoch 1/200
340/340 [==============================] - ETA: 0s - loss: 0.1471 - accuracy: 0.9429
Epoch 1: val_accuracy improved from -inf to 0.94694, saving model to /content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint_10.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


340/340 [==============================] - 124s 347ms/step - loss: 0.1471 - accuracy: 0.9429 - val_loss: 0.1461 - val_accuracy: 0.9469 - lr: 1.0000e-04
Epoch 2/200
340/340 [==============================] - ETA: 0s - loss: 0.1429 - accuracy: 0.9450
Epoch 2: val_accuracy improved from 0.94694 to 0.94842, saving model to /content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint_10.h5
340/340 [==============================] - 126s 370ms/step - loss: 0.1429 - accuracy: 0.9450 - val_loss: 0.1458 - val_accuracy: 0.9484 - lr: 1.0000e-04
Epoch 3/200
340/340 [==============================] - ETA: 0s - loss: 0.1459 - accuracy: 0.9428
Epoch 3: val_accuracy did not improve from 0.94842
340/340 [==============================] - 118s 346ms/step - loss: 0.1459 - accuracy: 0.9428 - val_loss: 0.1456 - val_accuracy: 0.9469 - lr: 1.0000e-04
Epoch 4/200
340/340 [==============================] - ETA: 0s - loss: 0.1428 - accuracy: 0.9432
Epoch 4: val_accuracy did not improve from 0.9

In [ ]:
model_path = '/content/drive/MyDrive/MobileNetV3_model_data/checkpoint_small/SGD_checkpoint.h5'############이전 데이터 한걸로 바꿔주기
base_model = tf.keras.models.load_model(model_path)
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Function  (None, 7, 7, 576)         939120    
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 576)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               73856     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 1014266 (3.87 MB)
Trainable params: 100215

In [ ]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small_input (In  [(None, 224, 224, 3)]     0         
 putLayer)                                                       
                                                                 
 MobilenetV3small (Function  (None, 7, 7, 576)         939120    
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 576)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               73856     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                           

## 음식 리스트 생성

In [ ]:
import os
import pandas as pd

# 기본 경로 설정
base_path = '/content/drive/MyDrive/food_data/Crop'

# 데이터를 저장할 리스트 초기화
data = []

# 상위 디렉토리 탐색
for sub_dir in os.listdir(base_path):
    sub_dir_path = os.path.join(base_path, sub_dir)
    if os.path.isdir(sub_dir_path):  # 상위 디렉토리 확인
        # 하위 디렉토리 탐색
        for menu_dir in os.listdir(sub_dir_path):
            menu_dir_path = os.path.join(sub_dir_path, menu_dir)
            if os.path.isdir(menu_dir_path):  # 메뉴 디렉토리 확인
                # 데이터 추가
                data.append({
                    'parent_directory': sub_dir,
                    'menu_name': menu_dir
                })

# 데이터프레임 생성
df = pd.DataFrame(data)

# CSV 파일로 저장
output_path = '/content/drive/MyDrive/MobileNetV3_model_data/menu_directories.csv'
df.to_csv(output_path, index=False)

print(f"CSV 파일이 성공적으로 저장되었습니다: {output_path}")


CSV 파일이 성공적으로 저장되었습니다: /content/drive/MyDrive/MobileNetV3_model_data/menu_directories.csv


In [ ]:
import os
import pandas as pd

# 루트 디렉토리 경로
root_dir = "/content/drive/MyDrive/food_data/Crop"

# 결과를 저장할 리스트
data = []

# 루트 디렉토리 아래의 모든 디렉토리 순회
for dir_number in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_number)
    if os.path.isdir(dir_path):
        for menu_name in os.listdir(dir_path):
            menu_path = os.path.join(dir_path, menu_name)
            if os.path.isdir(menu_path):
                data.append([dir_number, menu_name])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['디렉터리 번호', '메뉴 이름'])

# 선정한 100개의 음식 리스트
selected_foods = [
    "딸기", "떡국", "오이", "오이피클", "오징어", "가지", "가지구이", "감자구이", "감자샐러드",
    "감자스프", "잡채", "전복", "참치샌드위치", "콩국수", "토마토", "앵두", "야채볶음", "양배추샐러드",
    "양배추쌈", "양배추절임", "양상추샐러드", "양파", "양파샐러드", "어묵국", "애호박", "고구마",
    "고등어구이", "샌드위치", "순두부찌개", "순살치킨", "시루떡", "쌀국수", "아몬드", "아스파라거스구이",
    "알밥", "알타리김치", "애플민트", "오이김치", "옥수수스프", "완탕", "요거트", "연어구이", "연어샐러드",
    "연어초밥", "연어회", "오디", "오렌지", "아보카도샐러드", "아보카도", "아마씨", "쌀밥", "쌀",
    "시저샐러드", "적양배추", "자몽", "시금치", "브로콜리", "비빔밥", "메밀국수", "물미역", "미소장국",
    "미역나물", "후라이드치킨", "호박씨", "후랑크소시지", "피망", "우엉", "쑥갓", "상추", "새싹샐러드",
    "샐러리", "샐러드채소", "적채", "치커리", "카라멜", "마늘구이", "마늘장아찌", "김밥", "김치국",
    "김치볶음밥", "녹색피망", "단감", "단호박", "달걀", "달걀샌드위치", "달걀후라이", "두부", "된장국",
    "된장찌개", "메밀냉면", "보리빵", "복숭아", "샐러드", "산딸기", "살구", "사과", "사과샐러드",
    "사과파이", "살구파이", "블루베리"
]

# 선정한 음식만 필터링
filtered_df = df[df['메뉴 이름'].isin(selected_foods)]

# 필터링된 데이터를 새로운 CSV 파일로 저장
output_filtered_csv_path = "/content/drive/MyDrive/MobileNetV3_model_data/filtered_menu_directories.csv"
filtered_df.to_csv(output_filtered_csv_path, index=False, encoding='utf-8-sig')

print(f"CSV 파일이 성공적으로 저장되었습니다: {output_filtered_csv_path}")

CSV 파일이 성공적으로 저장되었습니다: /content/drive/MyDrive/MobileNetV3_model_data/filtered_menu_directories.csv
